In [ ]:
# The aim of this module is to calculate correlations for variables in the flat files
# the variables are then used for linear regression

# File name convention
# R_ = raw extract 
# E_ = Entry files into enrichment layer
# C_ = enriched layer with calculation
# I = insights layer, designed for model baselines
# V_ = validation

In [2]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

# I_FLAT = pd.read_pickle ('/Users/joezhou/Downloads/C_TICKER_FLAT.pkl') 
# C_BG_MODEL = pd.read_pickle ('/Users/joezhou/Downloads/C_BEN_G_MODEL.pkl') 

R_PRICE = pd.read_pickle ('/Users/joezhou/Downloads/Investment/1 data/R_ALL_Prices.pkl') 



In [ ]:
# Find daily price correlations

R_PRICE_CLSE = R_PRICE[['Date', 'TickName','Close']]
R_PRICE_CLSE2 = R_PRICE_CLSE.pivot_table(index=['Date'], columns='TickName', values='Close').reset_index()

# Create correlation matrix
corr_matrix = R_PRICE_CLSE2.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# upper.to_excel('/Users/joezhou/Downloads/Investment/corr_matrix.xlsx', engine='xlsxwriter')  
# Find features with correlation greater than threshold
to_drop = [column for column in upper.columns if any(upper[column] > 0.7)]
# Drop features 
R_PRICE_CLSE2.drop(to_drop, axis=1, inplace=True)


In [26]:

from datetime import datetime as dt
dte = dt.now().strftime('%Y%m%d')
fname = "/Users/joezhou/Downloads/Investment/TEST_CORR_MATRIX{}.xlsx".format(dte)
upper.to_excel(fname)

# last run, 1.41

In [62]:
# cluster to find similar performing

import matplotlib.pyplot as plt
# from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler


In [67]:
features, true_labels = make_blobs(
n_samples=200,
centers=3,
cluster_std=2.75,
random_state=42
 )

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

kmeans = KMeans(
   init="random",
   n_clusters=3,
   n_init=10,
   max_iter=300,
   random_state=42
   )


kmeans.fit(scaled_features)

KMeans(init='random', n_clusters=3, random_state=42)

In [ ]:
# Build sector related models
MOD_DATA = ADDED_PRICE_INF[['TickName','PROB_HIGH','CHG_RTE_PRICE','sector']]

# padding rows into columns of 1 or 0
# MOD_DATA = pd.concat([MOD_DATA, pd.get_dummies(MOD_DATA['sector'])], axis=1);


# transpose for excel movement correlations


MOD_DATA2 = MOD_DATA.pivot_table('PROB_HIGH', ['TickName','CHG_RTE_PRICE'], 'sector')

#Export version for Tableau

MOD_DATA2.to_excel('/Users/joezhou/Downloads/T_SECTOR_MODEL2.xls', index=False)


# del MOD_DATA['sector'], MOD_DATA['TickName']

# np.array(MOD_DATA, dtype=bool)


In [ ]:
# BG_MODEL.to_pickle("/Users/joezhou/Downloads/C_CALCS.pkl")
# BG_MODEL.to_excel('/Users/joezhou/Downloads/C_CALCS.xlsx', engine='xlsxwriter')  

